In [1]:
# check if notebook is in colab
try:
    # install ezkl
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


# Defines the model
# we got convs, we got relu, we got linear layers
# What else could one want ????

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=2, out_channels=3, kernel_size=5, stride=2)

        self.relu = nn.ReLU()

        self.d1 = nn.Linear(48, 48)
        self.d2 = nn.Linear(48, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim = 1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = self.relu(x)

        # logits => 32x10
        logits = self.d2(x)

        return logits


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




In [2]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.ezkl')
pk_path = os.path.join('pk.key')
vk_path = os.path.join('vk.key')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

sol_path = os.path.join("../src/verifierContract.sol")
abi_path = os.path.join("verifier_abi.json")

In [3]:

shape = [1, 28, 28]
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = 0.1*torch.rand(1,*shape, requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


In [5]:
# private input, public output, public params/model
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True


In [4]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

# Serialize data into file:
json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

calibration failed max lookup input (-1775279461, 1448637591) is too large
calibration failed max lookup input (-3447489334187, 2999935220695) is too large
calibration failed max lookup input (-3447489334187, 2999935220695) is too large
calibration failed max lookup input (-7100748157, 5799556778) is too large
calibration failed max lookup input (-27587429800270, 23997497660314) is too large
calibration failed max lookup input (-27587429800270, 23997497660314) is too large
calibration failed max lookup input (-40906178374, 42890034509) is too large
calibration failed max lookup input (-220682617436702, 191977434428592) is too large
calibration failed max lookup input (-220682617436702, 191977434428592) is too large
calibration failed max lookup input (-14202942623, 11597818937) is too large
calibration failed max lookup input (-55174781479162, 47996995103106) is too large
calibration failed max lookup input (-55174781479162, 47996995103106) is too large
Using 2 columns for non-linearit

True

In [7]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [8]:
# srs path
res = await ezkl.get_srs( settings_path, srs_path=srs_path)

In [10]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,

    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)
assert os.path.isfile(srs_path)

In [ ]:
# now generate the witness file

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [11]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,

        "single",
    )

print(res)
json.dump( res, open("proof.json", 'w' ))
assert os.path.isfile(proof_path)

{'instances': [['c501000000000000000000000000000000000000000000000000000000000000', '73feffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '5d00000000000000000000000000000000000000000000000000000000000000', '3a01000000000000000000000000000000000000000000000000000000000000', 'c600000000000000000000000000000000000000000000000000000000000000', '3101000000000000000000000000000000000000000000000000000000000000', 'fdfdffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', 'e3fdffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '7ffeffef93f5e1439170b97948e833285d588181b64550b829a031e1724e6430', '0d00000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x0643f3d6c182a60837167bdcf12884a42b2c7ca8642696a1d5901e90561ec86614747a063171b187adaf9168fbe524ad5e6081c772cc1aa54d277e18cebf0a892155318acc36fca134c102ffcc561e77db4fcc7efa81572cd2cd96321356ccd700d5b796b9f5023ef7594f12130aba9b3d7d21a8b31a77bcdc26539360640b1a1f7d54f26d532a5e10ef21b92aec2fc2ef

In [12]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        
    )

assert res == True
print("verified")

verified


In [18]:
# ezkl.create_evm_verifier(vk_path=Ellipsis, settings_path=Ellipsis, sol_code_path=Ellipsis, abi_path=Ellipsis, srs_path=None, render_vk_seperately=Ellipsis

res = await ezkl.create_evm_verifier(vk_path, settings_path, sol_path, abi_path)
assert res == True